In [1]:
#TFM UNIR BIG DATA

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("/kaggle/input/dataset/preus_tancament_r.csv")

In [4]:
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Unnamed: 0,Date,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted,IBEX.Open,IBEX.High,...,CABK.Low,CABK.Close,CABK.Volume,CABK.Adjusted,FER.Open,FER.High,FER.Low,FER.Close,FER.Volume,FER.Adjusted
0,1,2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,3.991400e+09,1132.989990,11986.500000,12145.099609,...,1.37222,1.37222,0.0,1.37222,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,2.491020e+09,1136.520020,12141.799805,12240.500000,...,1.32222,1.36667,833.0,1.36667,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,4.972660e+09,1137.140015,12216.400391,12230.700195,...,1.36111,1.38333,163881.0,1.38333,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,5.270680e+09,1141.689941,12163.000000,12199.700195,...,1.36667,1.37778,31284.0,1.37778,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,4.389590e+09,1144.979980,12218.000000,12232.200195,...,1.37778,1.37778,0.0,1.37778,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
assets = ['BBVA', 'TEF', 'GOOGL', 'IBEX', 'GSPC']

# Inicialitzar DataFrame de treball
df_features = df
df_features['Date'] = pd.to_datetime(df['Date'])

In [20]:

# Per a cada actiu, generem les següents variables:
# - Retorn diari
# - Target (1 si retorn demà és positiu, 0 si no)
# - Lags (retards dels retorns)
# - Mitjana mòbil i desviació típica (rolling)

for asset in assets:
    close_col = f'{asset}.Close'
    volume_col = f'{asset}.Volume'

    # Verificació per assegurar que existeixen les columnes
    if close_col in df.columns and volume_col in df.columns:
        # Retorn diari
        df_features[f'{asset}_return'] = df[close_col].pct_change(fill_method=None)

        # Variable objectiu: retorn de demà
        df_features[f'{asset}_target'] = (df_features[f'{asset}_return'].shift(-1) > 0).astype(int)

        # Lags
        df_features[f'{asset}_lag1'] = df_features[f'{asset}_return'].shift(1)
        df_features[f'{asset}_lag2'] = df_features[f'{asset}_return'].shift(2)

        # Mitjana mòbil i desviació
        df_features[f'{asset}_ma5'] = df[close_col].rolling(5).mean()
        df_features[f'{asset}_std5'] = df[close_col].rolling(5).std()

        # Volum (amb normalització per z-score)
        df_features[f'{asset}_volume_z'] = (df[volume_col] - df[volume_col].rolling(20).mean()) / df[volume_col].rolling(20).std()



/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


In [17]:
# Eliminar files amb NaN
df_clean = df_features.dropna().reset_index(drop=True)

# Mostrem les primeres files del DataFrame net
df_clean.head()

,Unnamed: 0,Date,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted,IBEX.Open,IBEX.High,...,IBEX_ma5,IBEX_std5,IBEX_volume_z,GSPC_return,GSPC_target,GSPC_lag1,GSPC_lag2,GSPC_ma5,GSPC_std5,GSPC_volume_z
0,2189,2018-06-25,2742.939941,2742.939941,2698.669922,2717.070068,3.663280e+09,2717.070068,9711.400391,9737.900391,...,9731.280078,72.985202,-0.817009,-0.013725,1,0.001862,-0.006346,2750.324023,19.787757,-0.138744
1,2190,2018-06-26,2722.120117,2732.909912,2715.600098,2723.060059,3.568900e+09,2723.060059,9670.299805,9699.799805,...,9707.680078,81.783502,-0.651848,0.002205,0,-0.013725,0.001862,2742.418018,21.485834,-0.273547
2,2191,2018-06-27,2728.449951,2746.090088,2699.379883,2699.629883,3.803990e+09,2699.629883,9636.599609,9723.900391,...,9681.619922,69.229075,-0.217679,-0.008604,1,0.002205,-0.013725,2728.879980,23.134845,0.078632
3,2192,2018-06-28,2698.689941,2724.340088,2691.989990,2716.310059,3.461100e+09,2716.310059,9603.599609,9669.200195,...,9659.000000,78.694643,-0.809613,0.006179,1,-0.008604,0.002205,2722.189990,20.243096,-0.406443


In [ ]:
#Cal tallar això per al codi


# Repetim l'execució anterior per tornar a obtenir els resultats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
import warnings

warnings.filterwarnings("ignore")

# Definim el grid de paràmetres
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}

# Preparem la validació creuada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Guardem els resultats
results = []

# Repetim per cada actiu
for asset in assets:
    target_col = f'{asset}_target'
    feature_cols = [col for col in df_clean.columns if col.startswith(asset) and 'target' not in col]

    if len(feature_cols) == 0 or target_col not in df_clean.columns:
        continue

    X = df_clean[feature_cols]
    y = df_clean[target_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    # Grid search amb Random Forest
    clf = RandomForestClassifier(random_state=42)
    grid = GridSearchCV(clf, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
    grid.fit(X_train, y_train)

    # Predictions
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]

    roc = roc_auc_score(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    results.append({
        'asset': asset,
        'roc_auc': roc,
        'accuracy': acc,
        'conf_matrix': cm,
        'best_params': grid.best_params_
    })

results
